- 해당 노트북 파일은 주피터 노트북을 통해 작성되었습니다.

### 1. 건물 정보 데이터 + 기타 데이터 불러오기

In [2]:
import pandas as pd

In [7]:
total = pd.read_csv('building_total.csv', encoding='cp949')

In [119]:
building_info = pd.read_csv('data_building_origin.csv', encoding='cp949')

C:\Users\KBM\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (35) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [22]:
total.head()

,도로명주소,위도,경도,haver_dis_subway,haver_dis_school,haver_dis_mart,mean_sales,mean_pop
0,서울특별시 강동구 양재대로 1540,37.542920,127.142450,0.316026,0.459123,0.110586,2.551361e+10,18225491.8
1,서울특별시 강동구 양재대로 1449,37.535248,127.138685,0.307749,0.416656,0.139283,4.715353e+10,20419730.5
2,서울특별시 강동구 양재대로128길,37.544508,127.144746,0.207319,0.498036,0.232098,2.896793e+10,22305878.2
3,서울특별시 강동구 천호대로 1156,37.533434,127.139603,0.487983,0.432536,0.346491,3.770563e+10,15761681.4
4,서울특별시 강동구 천호대로 1238,37.535540,127.148192,0.765237,0.442360,0.746099,3.966936e+10,12552664.0


In [23]:
building_info.head()

,NO,일반/집합 구분,대지위치,도로명주소,건물명칭,동명칭,세대,호,가구,대지면적(㎡),...,허가일자,착공일자,사용승인일자,에너지효율등급,에너지절감률(%),에너지성능지표(EPI)점수,친환경인증등급,친환경인증점수,지능형인증등급,지능형인증점수
0,1,일반,서울특별시 강동구 길동 336-1,서울특별시 강동구 양재대로 1540,NaN,NaN,0,0,0,1864.1,...,2004.6.11,2004.7.12,2004.9.24,NaN,0.0,0,NaN,0,NaN,0
1,2,일반,서울특별시 강동구 길동 413-45,서울특별시 강동구 양재대로 1449,NaN,NaN,0,0,0,1828.0,...,1999.7.27,1999.8.6,1999.11.6,NaN,0.0,73,NaN,0,NaN,0
2,3,집합,서울특별시 강동구 길동 54,서울특별시 강동구 양재대로128길,삼익파크,상가동,0,128,0,0.0,...,NaN,NaN,1982.9.4,NaN,0.0,0,NaN,0,NaN,0
3,4,일반,서울특별시 강동구 둔촌동 435-5,서울특별시 강동구 천호대로 1156,NaN,NaN,0,0,0,1274.3,...,2013.6.3,2014.1.9,2014.5.28,NaN,0.0,0,NaN,0,NaN,0
4,5,집합,서울특별시 강동구 둔촌동 603,서울특별시 강동구 천호대로 1238,프라자상가,NaN,0,41,0,0.0,...,1983.12.21,NaN,1984.12.10,NaN,0.0,0,NaN,0,NaN,0


### 2. 분석에 활용할 컬럼은 용도, 연면적, 층수, 주차장, 승강기를 선정했습니다.

In [120]:
building_info = building_info.loc[:,['용도', '연면적(㎡)', '지하층수(층)', '지상층수(층)', '주차장_자주식_옥내(대)',
               '주차장_자주식_옥외(대)', '주차장_자주식_인근(대)', '주차장_기계식_옥내(대)', '주차장_기계식_옥외(대)',
               '주차장_기계식_인근(대)', '승용승강기(대)', '비상용승강기(대)']]

- 건물별 생애이력 데이터와 앞서 구축한 데이터셋 결합

In [121]:
df = pd.concat([total, building_info], axis=1)

- 자주식 주차장과 기계식 주차장으로 옥내, 옥외, 인근 주차장 통합
- 승강기 + 비상용승강기 --> 승강기개수로 통합
- 지하층수 + 지상층수 --> 층수로 통합

In [122]:
df['자주식주차장'] = df['주차장_자주식_옥내(대)'] + df['주차장_자주식_옥외(대)'] + df['주차장_자주식_인근(대)']
df['기계식주차장'] = df['주차장_기계식_옥내(대)'] + df['주차장_기계식_옥외(대)'] + df['주차장_기계식_인근(대)']
df['승강기개수'] = df['승용승강기(대)'] + df['비상용승강기(대)']
df['층수'] = df['지하층수(층)'] + df['지상층수(층)']

In [123]:
df= df.drop(['주차장_자주식_옥내(대)', '주차장_자주식_옥외(대)', '주차장_자주식_인근(대)',
        '주차장_기계식_옥내(대)', '주차장_기계식_옥외(대)', '주차장_기계식_인근(대)',
        '승용승강기(대)', '비상용승강기(대)', '지하층수(층)', '지상층수(층)'], axis=1)

In [125]:
df.to_csv('data_before.csv', encoding='euc-kr', index=False)

In [124]:
df

,도로명주소,위도,경도,haver_dis_subway,haver_dis_school,haver_dis_mart,mean_sales,mean_pop,용도,연면적(㎡),자주식주차장,기계식주차장,승강기개수,층수
0,서울특별시 강동구 양재대로 1540,37.542920,127.142450,0.316026,0.459123,0.110586,2.551361e+10,18225491.8,판매시설,1543.47,21,0,0,2
1,서울특별시 강동구 양재대로 1449,37.535248,127.138685,0.307749,0.416656,0.139283,4.715353e+10,20419730.5,판매시설,2772.93,13,0,1,3
2,서울특별시 강동구 양재대로128길,37.544508,127.144746,0.207319,0.498036,0.232098,2.896793e+10,22305878.2,판매시설,3640.85,0,0,0,4
3,서울특별시 강동구 천호대로 1156,37.533434,127.139603,0.487983,0.432536,0.346491,3.770563e+10,15761681.4,판매시설,1625.67,16,0,1,3
4,서울특별시 강동구 천호대로 1238,37.535540,127.148192,0.765237,0.442360,0.746099,3.966936e+10,12552664.0,판매시설,1189.40,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119967,서울특별시 영등포구 영중로 155,37.530283,126.905429,0.571565,0.273858,0.283304,3.073583e+10,9318388.0,제2종근린생활시설,33.75,0,0,0,1
119968,서울특별시 영등포구 영중로 163,37.530840,126.905173,0.505860,0.252851,0.340109,3.073583e+10,9318388.0,제2종근린생활시설,1592.64,0,0,0,4
119969,서울특별시 영등포구 영중로 155-8,37.530245,126.904834,0.555776,0.315615,0.271392,3.073583e+10,9318388.0,제2종근린생활시설,302.40,0,0,0,5
119970,서울특별시 영등포구 영중로 157-11,37.530342,126.904867,0.546632,0.306686,0.282294,3.073583e+10,9318388.0,제2종근린생활시설,300.00,0,0,0,5


- 위와 같이 최종 데이터셋을 구축했습니다.